
# GPT Model



### Team S.A.M

#### Members  
- Akash Cuntur Shrinivasmurthy  
- Swathi Chandrashekaraiah 
- Madhukar Devendrappa  
- Bekzod Nazarov  

# Requirements  and imports

In [ ]:
# missing from docker image
!pip install kagglehub
!pip install torch
!pip install nltk
!pip install transformers

In [9]:
import os
import re
import math
import json
import pickle
import random
import kagglehub
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from collections import Counter
from collections import defaultdict
from transformers import GPT2Tokenizer
from datetime import datetime
import numpy as np
import pandas as pd

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Data Preprocessing

dataset is on [wine reviews](https://www.kaggle.com/datasets/zynicide/wine-reviews)

In [10]:
class WineReviewProcessor:
    def __init__(self, dataset_name="zynicide/wine-reviews"):
        self.dataset_name = dataset_name
        self.path = kagglehub.dataset_download(self.dataset_name)
        print("Path to dataset files:", self.path)
        self.files = [file for file in os.listdir(self.path) if os.path.isfile(os.path.join(self.path, file))]
        print("Dataset files:", self.files)
        self.data = pd.read_csv(f"{self.path}/winemag-data-130k-v2.csv")
        self.data.fillna("", inplace=True)
        print("Missing values:", self.data.isnull().sum())
        print("Number of rows:", self.data.shape[0])

        self.data["processed_text"] = self._process_text()

    def _process_text(self):
        """Preprocesses the dataset by combining relevant fields into a single text."""
        return (
            "wine review : " +
            self.data["country"].str.strip() + " : " +
            self.data["province"].str.strip() + " : " +
            self.data["variety"].str.strip() + " : " +
            self.data["description"].str.strip()
        )

    def get_processed_data(self):
        """Returns the processed text data."""
        return self.data["processed_text"].tolist()

    def get_sample(self, index):
        """Returns a sample of the processed text by index."""
        return self.data["processed_text"].iloc[index]

    def get_dataset_statistics(self):
        """Provides basic statistics about the dataset."""
        return {
            "total_rows": self.data.shape[0],
            "missing_values": self.data.isnull().sum().to_dict(),
            "unique_countries": self.data["country"].nunique(),
            "unique_provinces": self.data["province"].nunique(),
            "unique_varieties": self.data["variety"].nunique(),
        }

    def convert_to_lowercase(self):
        """Converts all characters in the processed text to lowercase."""
        self.data["processed_text"] = self.data["processed_text"].str.lower()

Number of rows: 129971

if rows with missing values were removed using data.dropna()
Number of rows: 22387

too less rows after dropping the rows with no data, so decided to fill the missing values insted

In [11]:
processor = WineReviewProcessor(dataset_name="zynicide/wine-reviews")
processor.convert_to_lowercase()
processed_texts = processor.get_processed_data()

100%|██████████| 50.9M/50.9M [00:12<00:00, 4.20MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4
Dataset files: ['winemag-data-130k-v2.json', 'winemag-data_first150k.csv', 'winemag-data-130k-v2.csv']
Missing values: Unnamed: 0               0
country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
dtype: int64
Number of rows: 129971


In [12]:
processed_texts[:3]

["wine review : italy : sicily & sardinia : white blend : aromas include tropical fruit, broom, brimstone and dried herb. the palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
 "wine review : portugal : douro : portuguese red : this is ripe and fruity, a wine that is smooth while still structured. firm tannins are filled out with juicy red berry fruits and freshened with acidity. it's  already drinkable, although it will certainly be better from 2016.",
 'wine review : us : oregon : pinot gris : tart and snappy, the flavors of lime flesh and rind dominate. some green pineapple pokes through, with crisp acidity underscoring the flavors. the wine was all stainless-steel fermented.']

# Tokenizer

In [13]:
def tokenize_text(text):
    """
    Tokenizes the text and creates a frequency dictionary of tokens.
    - Splits by spaces, hyphens, and special characters.
    - Replaces numbers with <NUM>.
    - Applies lemmatization to words.
    - Converts contractions like "isn't" to "isnt".
    - Removes unwanted characters.
    """
    # Create a dictionary to store token frequencies
    token_dict = defaultdict(int)

    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Define a regular expression that matches words, numbers, periods, and colons
    regex = r"[a-zA-Z]+|\d+|[.:]+"

    # Replace contractions with simplified forms
    text = re.sub(r"'", "", text)

    # Find all matches based on the regex
    tokens = re.findall(regex, text)

    # Loop through the tokens
    for token in tokens:
        token = token.lower()

        if token.isdigit():
            # Replace numbers with <NUM>
            token_dict["<NUM>"] += 1
        elif token in {".", ":"}:  # Keep periods and colons as is
            token_dict[token] += 1
        else:
            # Apply lemmatization to words
            lemmatized_token = lemmatizer.lemmatize(token)
            token_dict[lemmatized_token] += 1

    return token_dict

In [14]:
my_text = " : ".join(processed_texts)
my_text[:1000]

"wine review : italy : sicily & sardinia : white blend : aromas include tropical fruit, broom, brimstone and dried herb. the palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity. : wine review : portugal : douro : portuguese red : this is ripe and fruity, a wine that is smooth while still structured. firm tannins are filled out with juicy red berry fruits and freshened with acidity. it's  already drinkable, although it will certainly be better from 2016. : wine review : us : oregon : pinot gris : tart and snappy, the flavors of lime flesh and rind dominate. some green pineapple pokes through, with crisp acidity underscoring the flavors. the wine was all stainless-steel fermented. : wine review : us : michigan : riesling : pineapple rind, lemon pith and orange blossom start off the aromas. the palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish. : wine review : u

In [12]:
def save_vocab_to_json(token_dict, filename="vocab.json"):
    """
    Saves the vocabulary dictionary to a JSON file.
    The format will be:
    { 'vocab': [{'token': 'word', 'index': 0, 'count': 5}, ...] }
    """
    # Sort tokens by frequency in descending order
    sorted_tokens = sorted(token_dict.items(), key=lambda item: item[1], reverse=True)

    # Build the list of vocab entries with token, index, and count
    vocab_list = [{'token': str(token).strip(), 'index': idx, 'count': count} for idx, (token, count) in enumerate(sorted_tokens)]

    # Prepare the dictionary to be saved
    vocab_json = {'vocab': vocab_list}

    # Save to JSON file
    with open(filename, 'w') as f:
        json.dump(vocab_json, f, indent=4)

token_dict = tokenize_text(my_text.lower())
save_vocab_to_json(token_dict, "vocab.json")

In [15]:
class Tokenizer:
    def __init__(self, vocab_file="vocab.json", verbose=False):
        """
        Initializes the tokenizer by loading the vocab file.
        """
        self.vocab_file = vocab_file
        self.vocab = self.load_vocab()
        self.verbose = verbose
        self.token_to_id = {item['token']: item['index'] for item in self.vocab}
        self.id_to_token = {item['index']: item['token'] for item in self.vocab}
        self.lemmatizer = WordNetLemmatizer()

    def load_vocab(self):
        """Loads the vocabulary from the vocab file."""
        try:
            with open(self.vocab_file, 'r') as f:
                vocab_data = json.load(f)
                return vocab_data['vocab']
        except FileNotFoundError:
            raise FileNotFoundError(f"Vocabulary file '{self.vocab_file}' not found.")

    def encode(self, text):
        """Encodes the text into a list of token IDs."""
        # Define a regular expression that matches words, numbers, periods, and colons
        regex = r"[a-zA-Z]+|\d+|[.:]+"

        # Replace contractions with simplified forms (like "isn't" to "isnt")
        text = re.sub(r"'", "", text)

        # Find all matches based on the regex
        tokens = re.findall(regex, text)

        encoded = []

        # Loop through the tokens and process them
        for token in tokens:
            token = token.lower()

            if token.isdigit():
                # Replace numbers with <NUM>
                token = "<NUM>"
            elif token in {".", ":"}:  # Keep periods and colons as is
                pass
            else:
                # Apply lemmatization to words
                token = self.lemmatizer.lemmatize(token)

            # Check if the token is in the vocabulary
            if token in self.token_to_id:
                encoded.append(self.token_to_id[token])
            else:
                encoded.append(self.token_to_id.get("[UNK]", -1))
                if self.verbose:
                    print(f"Token -> {token} <- not found in vocab")

        return encoded

    def decode(self, encoded):
        """Decodes the list of token IDs back into a text string."""
        return ' '.join([self.id_to_token.get(idx, '[UNK]') for idx in encoded])

    def get_vocab_size(self):
        """Returns the size of the vocabulary."""
        return len(self.vocab)

    def limit_vocab_size(self, max_size):
        """
        Limits the vocabulary size to the most frequent `max_size` tokens.

        :param max_size: The maximum number of tokens to keep in the vocabulary.
        """
        # Sort the vocab by index (or frequency, if you store it)
        sorted_vocab = sorted(self.vocab, key=lambda x: x['index'])[:max_size]

        # Rebuild the token_to_id and id_to_token mappings
        self.vocab = sorted_vocab
        self.token_to_id = {item['token']: item['index'] for item in self.vocab}
        self.id_to_token = {item['index']: item['token'] for item in self.vocab}

        print(f"Vocabulary limited to {max_size} tokens.")
        print(f"New vocab size: {len(self.vocab)}")

In [16]:
coustom_tokenizer = Tokenizer(vocab_file="vocab.json")
text = "Hello mom! The tokenizer runs prety well for my GPT2 like model."

encode = coustom_tokenizer.encode(text)
print(encode)
print(len(encode))
print(coustom_tokenizer.decode(encode))

[12711, 10163, 3, -1, 759, -1, 65, 43, 4388, -1, 15, 110, 4944, 1]
14
hello mom the [UNK] run [UNK] well for my [UNK] <NUM> like model .


### In the end due to issues, decided to use GPT-2, BPE tokenizer

1. Custom tokenizer vocab count: 29262
2. gpt2 tokenizer vocab count: 50257

In [17]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token #to fix error with padding

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [24]:
encode = tokenizer.encode(text)
print(encode)
print(len(encode))
print(tokenizer.decode(encode))

[15496, 1995, 0, 383, 11241, 7509, 4539, 662, 774, 880, 329, 616, 402, 11571, 17, 588, 2746, 13]
18
Hello mom! The tokenizer runs prety well for my GPT2 like model.


# N-gram model

In [30]:
def generate_ngrams(text, n):
    words = text.split()
    ngrams = zip(*[words[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

def generate_text(context, ngram_counts, n, num_words):
    generated_text = context
    for _ in range(num_words):
        next_word = predict_next_word(generated_text, ngram_counts)
        if not next_word:
            break  # Stop if no prediction is possible
        generated_text += " " + next_word
    return generated_text

def predict_next_word(context, ngram_counts):
    context = context.strip().split()
    if len(context) < n - 1:
        raise ValueError(f"Context must have at least {n - 1} words for an {n}-gram model.")

    # Get the last n-1 words of the context
    context_key = " ".join(context[-(n - 1):])

    # Find all n-grams that start with the context
    candidates = {ngram: count for ngram, count in ngram_counts.items() if ngram.startswith(context_key)}

    if not candidates:
        return None  # No prediction possible

    total_count = sum(candidates.values())
    probabilities = {ngram: count / total_count for ngram, count in candidates.items()}

    # Adding repoducibility
    random.seed(13)
    next_word = random.choices(
        population=[ngram.split()[-1] for ngram in candidates.keys()],
        weights=probabilities.values()
    )[0]

    return next_word

## Tri-gram

In [38]:
n = 3
all_ngrams = []
for text in processed_texts:
    all_ngrams.extend(generate_ngrams(text, n))

ngram_counts = Counter(all_ngrams)

print(f"Top 10 most common {n}-grams:")
print(ngram_counts.most_common(10))

Top 10 most common 3-grams:
[('wine review :', 129971), ('review : us', 54504), (': us :', 54504), ('us : california', 36247), (': california :', 36247), ('review : france', 22093), (': france :', 22093), ('review : italy', 19540), (': italy :', 19540), ('red blend :', 17458)]


In [40]:
context = "wine review : portugal : douro :"
print(context)
# Number of words to predict
num_words = 30
print(generate_text(context, ngram_counts, n, num_words))

wine review : portugal : douro :
wine review : portugal : douro : portuguese red : this is a fine example of the best wines it was aged in 30% new french oak the this and make for a 3–4 years screwcap.


## Bi-gram

In [41]:
n = 2
all_ngrams = []
for text in processed_texts:
    all_ngrams.extend(generate_ngrams(text, n))

ngram_counts = Counter(all_ngrams)

num_words = 30

print(f"Top 10 most common {n}-grams:")
print(ngram_counts.most_common(10))
print("\n")
print(generate_text(context, ngram_counts, n, num_words))

Top 10 most common 2-grams:
[('wine review', 129971), ('review :', 129971), (': us', 54504), ('us :', 54504), (': california', 36257), ('california :', 36247), ('on the', 33195), (': this', 28766), ('blend :', 25730), ('this is', 24744)]


wine review : portugal : douro : pinot noir : pinot noir : pinot noir : pinot noir : pinot noir : pinot noir : pinot noir : pinot noir : pinot noir : pinot noir :


## 4-gram

In [42]:
n = 4
all_ngrams = []
for text in processed_texts:
    all_ngrams.extend(generate_ngrams(text, n))

ngram_counts = Counter(all_ngrams)

num_words = 30

print(f"Top 10 most common {n}-grams:")
print(ngram_counts.most_common(10))
print("\n")
print(generate_text(context, ngram_counts, n, num_words))

Top 10 most common 4-grams:
[('wine review : us', 54504), ('review : us :', 54504), (': us : california', 36247), ('us : california :', 36247), ('wine review : france', 22093), ('review : france :', 22093), ('wine review : italy', 19540), ('review : italy :', 19540), (': pinot noir :', 13272), (': cabernet sauvignon :', 9472)]


wine review : portugal : douro : portuguese red : this is a great wine for a drink window


# GPT Architecture

Model 1 : 11.6 million parameters
Model 2 : 13.9 million parameters
For reference, GPT-2 : 124 million parameters

### Method 1. Following this [medium](https://medium.com/@vipul.koti333/from-theory-to-code-step-by-step-implementation-and-code-breakdown-of-gpt-2-model-7bde8d5cecda) article

In [43]:
GPT_CONFIG_124M = {
    "vocab_size": 16384,
    "context_length": 64,
    "emb_dim": 256,
    "batch_size": 8,
    "n_heads": 4,
    "n_layers": 4,
    "drop_rate": 0.1,
    "qkv_bias": True
}

Hyperparameters used in the model:

- **`vocab_size`**: Defines the vocabulary size, which in our case is 16,384 tokens.
- **`context_length`**: Represents the model's maximum input token length, constrained by the positional embeddings.
- **`emb_dim`**: Refers to the embedding size for token inputs, where each input token is converted into a 256-dimensional vector.
- **`n_heads`**: Specifies the number of attention heads in the multi-head attention mechanism.
- **`n_layers`**: Indicates the number of transformer blocks in the model, which will be implemented in the following sections.
- **`drop_rate`**: Determines the intensity of the dropout mechanism, with a value of 0.1 implying that 10% of hidden units are dropped during training to reduce overfitting.
- **`qkv_bias`**: Controls whether a bias vector is added in the Linear layers of the multi-head attention mechanism when computing the query (Q), key (K), and value (V) tensors.



#### **GPT Model Breakdown:**

1. **Token Embedding**
2. **Positional Encoding**
3. **Transformer Blocks**</br>
   -a. Multi-Head Attention  
   -b. Layer Normalization  
   -c. Feed-Forward Neural Network
4. **Output Layer**

In [4]:
# 1. Embedding Layer
class Embedding(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

    def forward(self, x):
        return self.embed(x)

embedding = Embedding(GPT_CONFIG_124M["vocab_size"], GPT_CONFIG_124M["emb_dim"])

input_ids = torch.randint(0, GPT_CONFIG_124M["vocab_size"], (2, 10))

embed_output = embedding(input_ids)

print(f"Embedding output shape: {embed_output.shape}")

assert embed_output.shape == (2, 10, GPT_CONFIG_124M["emb_dim"]), "Embedding shape mismatch"

Embedding output shape: torch.Size([2, 10, 256])


In [5]:
# 2. Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_seq_length):
        super().__init__()
        pe = torch.zeros(max_seq_length, embed_size)

        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * -(math.log(10000.0) / embed_size))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        if torch.isnan(x).any():
            print("NaN detected in input embeddings!")

        pos_encoded_x = x + self.pe[:, :x.size(1)]

        if torch.isnan(pos_encoded_x).any():
            print("NaN detected in positional encoding output!")

        return pos_encoded_x


pos_encoding = PositionalEncoding(GPT_CONFIG_124M["emb_dim"], GPT_CONFIG_124M["context_length"])

pos_output = pos_encoding(embed_output)

print(f"Positional Encoding output shape: {pos_output.shape}")

assert pos_output.shape == embed_output.shape, "Positional Encoding shape mismatch"

Positional Encoding output shape: torch.Size([2, 10, 256])


In [6]:
# A. Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads, qkv_bias=False):
        super().__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads

        self.query = nn.Linear(embed_size, embed_size, bias=qkv_bias)
        self.key = nn.Linear(embed_size, embed_size, bias=qkv_bias)
        self.value = nn.Linear(embed_size, embed_size, bias=qkv_bias)
        self.out = nn.Linear(embed_size, embed_size)

    def forward(self, x, mask=None):
        batch_size = x.shape[0]

        q = self.query(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        attention = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(self.head_dim)
        if mask is not None:
            attention = attention.masked_fill(mask == 0, float('-inf'))
        attention = torch.softmax(attention, dim=-1)

        out = torch.matmul(attention, v)
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.embed_size)
        return self.out(out)

mha = MultiHeadAttention(GPT_CONFIG_124M["emb_dim"], GPT_CONFIG_124M["n_heads"])
mha_output = mha(pos_output)
print(f"Multi-Head Attention output shape: {mha_output.shape}")
assert mha_output.shape == pos_output.shape, "Multi-Head Attention shape mismatch"

Multi-Head Attention output shape: torch.Size([2, 10, 256])


In [7]:
# 4. Layer Normalization
class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)

        norm_x = (x - mean) / torch.sqrt(var + self.eps)

        return self.scale * norm_x + self.shift

emb_dim = GPT_CONFIG_124M["emb_dim"]
layer_norm = LayerNorm(emb_dim)

batch_size = 2
seq_length = 10
input_tensor = torch.randn(batch_size, seq_length, emb_dim)

ln_output = layer_norm(input_tensor)

print(f"LayerNorm output shape: {ln_output.shape}")
assert ln_output.shape == input_tensor.shape, "LayerNorm output shape mismatch"

output_mean = ln_output.mean(dim=-1)
output_var = ln_output.var(dim=-1, unbiased=False)

assert torch.allclose(output_mean, torch.zeros(batch_size, seq_length), atol=1e-6), "Mean is not close to 0"

assert torch.allclose(output_var, torch.ones(batch_size, seq_length), atol=1e-2), "Variance is not close to 1"

print("LayerNorm test passed!")

LayerNorm output shape: torch.Size([2, 10, 256])
LayerNorm test passed!


In [8]:
# 5. Feed-Forward Network
class FeedForward(nn.Module):
    def __init__(self, embed_size, ff_hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(embed_size, ff_hidden_size)
        self.fc2 = nn.Linear(ff_hidden_size, embed_size)
        self.gelu = nn.GELU()

    def forward(self, x):
        return self.fc2(self.gelu(self.fc1(x)))

ff_hidden_size = GPT_CONFIG_124M["emb_dim"] * 4
ff = FeedForward(GPT_CONFIG_124M["emb_dim"], ff_hidden_size)

ff_output = ff(mha_output)

print(f"Feed-Forward output shape: {ff_output.shape}")

assert ff_output.shape == mha_output.shape, "Feed-Forward shape mismatch"

Feed-Forward output shape: torch.Size([2, 10, 256])


In [9]:
# 6. Combined Transformer Block
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads, ff_hidden_size, dropout=0.1, qkv_bias=False):
        super().__init__()
        self.mha = MultiHeadAttention(embed_size, num_heads, qkv_bias)
        self.ff = FeedForward(embed_size, ff_hidden_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attention_output = self.ln1(x + self.dropout(self.mha(x, mask)))
        ff_output = self.ln2(attention_output + self.dropout(self.ff(attention_output)))
        return ff_output

transformer = TransformerBlock(GPT_CONFIG_124M["emb_dim"], GPT_CONFIG_124M["n_heads"], ff_hidden_size)

transformer_output = transformer(pos_output)

print(f"Transformer Block output shape: {transformer_output.shape}")

assert transformer_output.shape == pos_output.shape, "Transformer Block shape mismatch"

Transformer Block output shape: torch.Size([2, 10, 256])


In [10]:
# 7. GPT-2 Model
class GPT2(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embedding = Embedding(config["vocab_size"], config["emb_dim"])

        self.positional_encoding = PositionalEncoding(config["emb_dim"], config["context_length"])

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(config["emb_dim"], config["n_heads"], config["emb_dim"] * 4, config["drop_rate"], config["qkv_bias"])
            for _ in range(config["n_layers"])
        ])

        self.fc_out = nn.Linear(config["emb_dim"], config["vocab_size"])

        self.dropout = nn.Dropout(config["drop_rate"])

    def forward(self, x, mask=None):
        x = self.dropout(self.positional_encoding(self.embedding(x)))

        for block in self.transformer_blocks:
            x = block(x, mask)

        return self.fc_out(x)


model = GPT2(GPT_CONFIG_124M)

input_ids = torch.randint(0, GPT_CONFIG_124M["vocab_size"], (2, 64))

output = model(input_ids)

print(f"GPT-2 Model output shape: {output.shape}")

assert output.shape == (2, 64, GPT_CONFIG_124M["vocab_size"]), "GPT-2 Model shape mismatch"

GPT-2 Model output shape: torch.Size([2, 64, 16384])


In [ ]:
class GPT2Dataset(Dataset):
    def __init__(self, tokenized_text, seq_length):
        self.tokens = tokenized_text.squeeze(0)
        self.seq_length = seq_length

    def __len__(self):
        return len(self.tokens) // self.seq_length

    def __getitem__(self, idx):
        start = idx * self.seq_length
        end = start + self.seq_length
        input_ids = self.tokens[start:end]
        target_ids = input_ids.clone()
        return input_ids, target_ids

In [ ]:
class ChunkedDataset(Dataset):
    def __init__(self, tokenized_text, context_length):
        """
        Initializes the dataset with tokenized text and context length.
        tokenized_text: List of tokenized text (list of lists of token IDs).
        context_length: The length of each chunk.
        """
        self.tokens = tokenized_text
        self.context_length = context_length

    def __len__(self):
        total_tokens = sum(len(text) for text in self.tokens)
        return total_tokens // self.context_length

    def __getitem__(self, idx):
        current_token_index = idx * self.context_length
        chunk = []
        current_text_index = 0

        while len(chunk) < self.context_length and current_text_index < len(self.tokens):
            text = self.tokens[current_text_index]
            start = current_token_index - sum(len(t) for t in self.tokens[:current_text_index])
            end = min(len(text), start + (self.context_length - len(chunk)))

            chunk.extend(text[start:end])  # Add tokens from the current sequence
            current_token_index += end - start
            current_text_index += 1

        padding_length = self.context_length - len(chunk)
        if padding_length > 0:
            chunk = chunk + [0] * padding_length
        elif padding_length < 0:
            chunk = chunk[:self.context_length]

        return torch.tensor(chunk), torch.tensor(chunk)


In [ ]:
from torch.utils.data import random_split

tokenized_input = tokenizer.encode(joined_string)
tokenized_text = [tokenized_input[i:i + GPT_CONFIG_124M["context_length"]] for i in range(0, len(tokenized_input), GPT_CONFIG_124M["context_length"])]

# Split dataset into training and validation (e.g., 90% train, 10% validation)
train_size = int(0.9 * len(tokenized_text))
val_size = len(tokenized_text) - train_size

train_dataset = ChunkedDataset(tokenized_text[:train_size], GPT_CONFIG_124M["context_length"])
val_dataset = ChunkedDataset(tokenized_text[train_size:], GPT_CONFIG_124M["context_length"])

train_dataloader = DataLoader(train_dataset, batch_size=GPT_CONFIG_124M["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=GPT_CONFIG_124M["batch_size"], shuffle=False)


In [ ]:

model = GPT2(GPT_CONFIG_124M)
device = torch.device("cuda")
model.to(device)

# optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

run_name = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"./logs/{run_name}")  # Create a log directory for TensorBoard

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Training epoch {epoch + 1}...")
    model.train()
    total_train_loss = 0

    # Training phase
    for batch_idx, (input_ids, targets) in enumerate(train_dataloader):
        input_ids, targets = input_ids.to(device), targets.to(device)

        outputs = model(input_ids)

        logits = outputs.view(-1, GPT_CONFIG_124M["vocab_size"])  # Flatten for CrossEntropyLoss
        targets = targets.view(-1)  # Flatten targets

        loss = criterion(logits, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

        if (batch_idx + 1) % 1000 == 0:  # Log every 1000 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_dataloader)}], Loss: {loss.item():.6f}")

        writer.add_scalar('Loss/train', loss.item(), epoch * len(train_dataloader) + batch_idx)

    print(f"Epoch [{epoch + 1}/{num_epochs}] finished. Average Training Loss: {total_train_loss / len(train_dataloader):.4f}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch_idx, (input_ids, targets) in enumerate(val_dataloader):
            input_ids, targets = input_ids.to(device), targets.to(device)

            outputs = model(input_ids)

            logits = outputs.view(-1, GPT_CONFIG_124M["vocab_size"])  # Flatten for CrossEntropyLoss
            targets = targets.view(-1)  # Flatten targets

            loss = criterion(logits, targets)
            total_val_loss += loss.item()

        writer.add_scalar('Loss/val', total_val_loss / len(val_dataloader), epoch)

    print(f"Epoch [{epoch + 1}/{num_epochs}] finished. Average Validation Loss: {total_val_loss / len(val_dataloader):.4f}")

writer.close()

print("Training completed.")

In [ ]:
model_save_path = "gpt2_medium.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

#### **Generate output using GPT-2 Model**

In [16]:
GPT_CONFIG_124M = {'vocab_size': 16384,
 'context_length': 128,
 'emb_dim': 256,
 'batch_size': 16,
 'n_heads': 4,
 'n_layers': 4,
 'drop_rate': 0.1,
 'qkv_bias': True}

model = GPT2(GPT_CONFIG_124M)

checkpoint_path = "./checkpoint/model1/gpt2_medium.pth"

tokenizer = Tokenizer()
tokenizer.limit_vocab_size(GPT_CONFIG_124M["vocab_size"])

model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
model.eval()

Vocabulary limited to 16384 tokens.
New vocab size: 16384


<ipython-input-16-369f1b5616a1>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))


GPT2(
  (embedding): Embedding(
    (embed): Embedding(16384, 256)
  )
  (positional_encoding): PositionalEncoding()
  (transformer_blocks): ModuleList(
    (0-3): 4 x TransformerBlock(
      (mha): MultiHeadAttention(
        (query): Linear(in_features=256, out_features=256, bias=True)
        (key): Linear(in_features=256, out_features=256, bias=True)
        (value): Linear(in_features=256, out_features=256, bias=True)
        (out): Linear(in_features=256, out_features=256, bias=True)
      )
      (ff): FeedForward(
        (fc1): Linear(in_features=256, out_features=1024, bias=True)
        (fc2): Linear(in_features=1024, out_features=256, bias=True)
        (gelu): GELU(approximate='none')
      )
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (fc_out): Linear(in_features=256, out_features=16384, bias=True)
  (dropout): Dropout(p=0.1,

In [17]:
def generate_text_simple(model, idx, max_new_tokens, context_size, temperature=1.0):
    """
    Generate text using the provided model by predicting the next tokens based on the current context.

    Parameters:
    - model: The language model used for generating text.
    - idx: A tensor of shape (batch, n_tokens) containing the current context token indices.
    - max_new_tokens: The maximum number of new tokens to generate.
    - context_size: The maximum number of tokens to consider from the context.
    - temperature: A value to scale logits for controlling randomness in sampling.

    Returns:
    - idx: The updated tensor containing the original context and the newly generated tokens.
    """

    idx = idx.to(next(model.parameters()).device)
    for _ in range(max_new_tokens):

        idx_cond = idx[:, -context_size:]

        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]

        logits = logits / temperature

        probas = torch.softmax(logits, dim=-1)

        idx_next = torch.multinomial(probas, num_samples=1)

        idx = torch.cat((idx, idx_next), dim=1)

    return idx

In [21]:
start_context = "wine review : Italy : Oregon : Pinot Gris : "

encoded = tokenizer.encode(start_context)
print("Encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("Encoded tensor:", encoded_tensor)

model.eval()

out = generate_text_simple(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=50,
    context_size=GPT_CONFIG_124M["context_length"],
    temperature=3.0
)

print("Output:", out)
print("Output length:", len(out[0]))

decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print("Decoded text:", decoded_text)

Encoded: [4, 7, 0, 36, 0, 148, 0, 39, 392, 0]
Encoded tensor: tensor([[  4,   7,   0,  36,   0, 148,   0,  39, 392,   0]])
Output: tensor([[    4,     7,     0,    36,     0,   148,     0,    39,   392,     0,
          2866,  1501,   157, 15625,  3638,  7691,  3673,  7882,  3945, 16270,
          7090, 11205, 13453, 11120,  7384, 13851,  9777,  6449, 10386, 11921,
          1322, 15718,  5828,  3369,  2959, 15266, 15780, 13411, 10545,  3665,
         10019, 11417,  6928,  9200,  4681,   472,  3130, 12524,  6104,  3495,
          9552,   758, 12166, 10901, 10005, 12142, 10655, 14186,  1568,  5046]])
Output length: 60
Decoded text: wine review : italy : oregon : pinot gris : indication exhibit taste afficionados select cigarette awesome reluctant rotgipfler trine trending fined cerretta gassiness responsible bechthold shuttered vegetarian broker countywide immediate tinker amity history basically tertre vineam ferreirinha dampened diesel compels susanna davis specked steer pleasant cont

### Method 2: Using GPT2 tokenizer

In [22]:
# Adjusted hyperparameters
BATCH_SIZE = 48
BLOCK_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
D_MODEL = 384
N_HEAD = 6
N_LAYER = 6
DROPOUT = 0.2

#### Model architecture


In [ ]:
class TextDataset(Dataset):
    def __init__(self, text_data, block_size, tokenizer):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.examples = []

        for text in text_data:
            encodings = tokenizer(text, truncation=True, max_length=block_size,
                                padding='max_length', return_tensors="pt")

            if encodings['input_ids'].size(1) > 0:
                self.examples.append({
                    'input_ids': encodings['input_ids'][0],
                    'attention_mask': encodings['attention_mask'][0]
                })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return {
            "input_ids": self.examples[idx]['input_ids'],
            "attention_mask": self.examples[idx]['attention_mask'],
            "labels": self.examples[idx]['input_ids'].clone()
        }


In [23]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size, d_model, block_size, n_head, n_layer, dropout):
        super().__init__()
        self.block_size = block_size

        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(block_size, d_model)
        self.dropout = nn.Dropout(dropout)

        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=n_head,
                dim_feedforward=4 * d_model,
                dropout=dropout,
                batch_first=True,
                norm_first=True
            )
            for _ in range(n_layer)
        ])

        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.ones_(module.weight)
            torch.nn.init.zeros_(module.bias)

    def forward(self, idx, attention_mask=None, labels=None):
        B, T = idx.shape

        tok_emb = self.token_embedding(idx)
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.position_embedding(pos)

        x = self.dropout(tok_emb + pos_emb)

        if attention_mask is not None:
            mask = attention_mask.bool()
        else:
            mask = None

        for block in self.blocks:
            x = block(x, src_key_padding_mask=~mask if mask is not None else None)

        x = self.ln_f(x)
        logits = self.lm_head(x)

        if labels is None:
            return logits
        else:
            if attention_mask is not None:
                labels = labels.masked_fill(~attention_mask.bool(), -100)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)),
                                 labels.view(-1),
                                 ignore_index=-100)
            return logits, loss

    def generate(self, prefix, max_length, tokenizer, temperature=0.7, top_k=50):
        self.eval()
        with torch.no_grad():
            prefix_tokens = tokenizer.encode(prefix, return_tensors="pt").to(DEVICE)
            generated = prefix_tokens.clone()

            for _ in range(max_length):
                if generated.size(1) >= self.block_size:
                    generated = generated[:, -self.block_size:]

                logits = self(generated)
                next_token_logits = logits[:, -1, :] / temperature

                top_k_logits, top_k_indices = torch.topk(next_token_logits, k=top_k)
                probs = F.softmax(top_k_logits, dim=-1)
                next_token_idx = torch.multinomial(probs, num_samples=1)
                next_token = top_k_indices.gather(-1, next_token_idx)

                generated = torch.cat([generated, next_token], dim=1)

                if next_token.item() == tokenizer.eos_token_id:
                    break

            return tokenizer.decode(generated[0])


In [ ]:
def train_model(model, train_loader, val_loader, optimizer, scheduler, writer, epoch, save_path):
    model.train()
    total_loss = 0

    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        logits, loss = model(input_ids, attention_mask=attention_mask, labels=labels)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        if batch_idx % 100 == 0:
            writer.add_scalar('Training Loss', loss.item(), epoch * len(train_loader) + batch_idx)
            print(f'Epoch {epoch}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')

    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)

            logits, loss = model(input_ids, attention_mask=attention_mask, labels=labels)
            total_val_loss += loss.item()

            if batch_idx % 50 == 0:
                writer.add_scalar('Validation Loss', loss.item(), epoch * len(val_loader) + batch_idx)

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f}')

    checkpoint_path = f"{save_path}/epoch_{epoch}.pt"
    torch.save(model.state_dict(), checkpoint_path)
    print(f'Model checkpoint saved to {checkpoint_path}')

    return avg_train_loss, avg_val_loss

In [ ]:
processor = WineReviewProcessor(dataset_name="zynicide/wine-reviews")
processor.convert_to_lowercase()
text_data = processor.get_processed_data()

In [25]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# split ratio 80% for training, 20% for validation
train_ratio = 0.8
val_ratio = 0.2

dataset = TextDataset(text_data, BLOCK_SIZE, tokenizer)

total_size = len(dataset)
train_size = int(total_size * train_ratio)
val_size = total_size - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = GPTLanguageModel(
    vocab_size=tokenizer.vocab_size,
    d_model=D_MODEL,
    block_size=BLOCK_SIZE,
    n_head=N_HEAD,
    n_layer=N_LAYER,
    dropout=DROPOUT
).to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS)
writer = SummaryWriter('runs/gpt_training_now')

In [ ]:
save_path = "./checkpoints"
for epoch in range(EPOCHS):
    train_loss, val_loss = train_model(model, train_loader, val_loader, optimizer, scheduler, writer, epoch, save_path)
    print(f"Epoch {epoch} - Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

In [ ]:
model_save_path = "best.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

#### Generating output

In [29]:
BATCH_SIZE = 48
BLOCK_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
D_MODEL = 384
N_HEAD = 6
N_LAYER = 6
DROPOUT = 0.2

model = GPTLanguageModel(
    vocab_size=tokenizer.vocab_size,
    d_model=D_MODEL,
    block_size=BLOCK_SIZE,
    n_head=N_HEAD,
    n_layer=N_LAYER,
    dropout=DROPOUT
).to(DEVICE)
checkpoint_path = "./checkpoint/model2/epoch_3.pt"

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE))

model.eval()

<ipython-input-29-5857eca08a1b>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE))


GPTLanguageModel(
  (token_embedding): Embedding(50257, 384)
  (position_embedding): Embedding(256, 384)
  (dropout): Dropout(p=0.2, inplace=False)
  (blocks): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
      )
      (linear1): Linear(in_features=384, out_features=1536, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (linear2): Linear(in_features=1536, out_features=384, bias=True)
      (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.2, inplace=False)
      (dropout2): Dropout(p=0.2, inplace=False)
    )
  )
  (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=384, out_features=50257, bias=True)
)

In [58]:
test_prefix = "wine review : Italy : Oregon : Pinot Gris : A wine "
generated_text = model.generate(test_prefix, max_length=20, tokenizer=tokenizer, temperature=10.0, top_k=50)
print(f"Generated text:\n{generated_text}")

Generated text:
wine review : Italy : Oregon : Pinot Gris : A wine  about preserves complexity barrel complexity complexitygoing preserves juinesionalional working z ahead whisper high works thinktext


# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./runs